In [ ]:
!wget https://hf.co/danjacobellis/walloc/resolve/main/Stereo_Li_108c_test2.pth

In [ ]:
!wget https://huggingface.co/danjacobellis/LCCL/resolve/main/vss_walloc_5x.pth

In [13]:
import io
import torch
import torchaudio
import torchvision
import numpy as np
from datasets import load_dataset
from walloc import walloc
from IPython.display import display, Audio, Image, update_display, HTML
from torchvision.transforms import ToPILImage
from fastprogress.fastprogress import master_bar, progress_bar
from ioae import IsotropicOobleckAutoencoder
from spauq.core.metrics import spauq_eval
from einops import rearrange
import cdpam
class Config: pass

In [14]:
def compress(x_valid, codec):
    compressed_data = codec.encoder(codec.wavelet_analysis(x_valid / 2, J=codec.J))
    return compressed_data

def decompress(compressed_data, codec):
    decompressed_data = 2*codec.clamp(codec.post(codec.wavelet_synthesis(codec.decoder(compressed_data), codec.J)))
    return decompressed_data

In [16]:
device = "cuda"

cdpam_loss = cdpam.CDPAM()

codec_checkpoint = torch.load("Stereo_Li_108c_test2.pth",map_location="cpu",weights_only=False)
codec_config = codec_checkpoint['config']
codec = walloc.Codec1D(
    channels = codec_config.channels,
    J = codec_config.J,
    Ne = codec_config.Ne,
    Nd = codec_config.Nd,
    latent_dim = codec_config.latent_dim,
    latent_bits = codec_config.latent_bits,
    lightweight_encode = codec_config.lightweight_encode,
    post_filter = codec_config.post_filter
).to(device)
codec.load_state_dict(codec_checkpoint['model_state_dict'])
codec.eval();

checkpoint = torch.load("vss_walloc_5x.pth",map_location="cpu",weights_only=False)
config = checkpoint['config']
model = IsotropicOobleckAutoencoder(
    channels=codec_config.latent_dim,
    patch_size=config.patch_size,
    embed_dim=config.embed_dim,
    depth=config.depth,
).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
codec.load_state_dict(checkpoint['codec_state_dict']) # post filter
model.eval()
sum(p.numel() for p in model.parameters())/1e6

6.8364

In [17]:
valid_dataset = load_dataset("danjacobellis/musdb18hq_vss",split='validation')

In [50]:
with torch.no_grad():
    sample = valid_dataset[200]
    xb, fs = torchaudio.load(sample['audio_mix']['bytes'])
    vb, fs = torchaudio.load(sample['audio_vocal']['bytes'])
    B = xb.shape[-1]//config.length_samples
    x = torch.zeros((B,2,config.length_samples))
    v = torch.zeros((B,2,config.length_samples))
    for i_frame in range(B):
        i_start = i_frame*config.length_samples
        i_end = (i_frame+1)*config.length_samples
        x[i_frame,:,:] = xb[:,i_start:i_end]
        v[i_frame,:,:] = vb[:,i_start:i_end]
    x = x.to(device)
    v = v.to(device)
    xc = compress(x, codec)
    pred = model(xc)
    v_hat = decompress(pred,codec)

    v = rearrange(v, 'B C L -> C (B L)')/2
    v_hat = rearrange(v_hat, 'B C L -> C (B L)')/2
    
    PSNR = -10*np.log10(torch.nn.functional.mse_loss(v,v_hat).item())
    SDR = spauq_eval(v.cpu(),v_hat.cpu(),fs=fs)
    SSDR = SDR['SSR']
    SRDR = SDR['SRR']
    cdpam = cdpam_loss.forward(v,v_hat).mean().item()

In [52]:
print(PSNR)
print(SSDR)
print(SRDR)
print(-10*np.log10(cdpam))

31.725501225961295
6.367874199474555
0.06404294549041384
36.51595032171185
